![BridgingAI Logo](../bridgingai_logo.png)

# BasicsML - Exercise 4: Linear Regression
In this exercise, you will implement linear regression using the least-squares formulation as well as ridge regression for improved robustness against overfitting.

Make sure to replace all parts that say
```python
# YOUR CODE HERE
raise NotImplementedError()
```

Happy coding!

# Q1: Implement least squares regression
In this first part of the exercise, you implement the least-squares solution to linear regression.

**Hint:** this is very similar to training a least-squares classifier, so you might be able to reuse code from that exercise.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from functools import partial

In [ ]:
def load_data(name):
    data = np.load(f"{name}.npz")
    return tuple(data[f] for f in data.files)

In [ ]:
def plot_(data, label, params=None, basis_fun=None):
    # Plot the data points and the regression function
    plt.subplot()

    plt.scatter(data.T, label.T, c="blue", marker="x")

    if params:
        xmin, xmax = plt.xlim()
        x = np.linspace(xmin, xmax, 200)[..., None]
        fx = linreg(*params, x, basis_fun)
        plt.plot(x, fx, c="orange")

    plt.show()

Implement the `least_squares` function that computes the least-squares solution given some labelled data.
Afterwards, complete the `linreg` function; this function should apply a given linear regression model on some data.
It optionally transforms the data using a basis function, if provided.

In [ ]:
def least_squares(data, label):
    # computes the parameters of a least-squares regression model
    # Input
    #  data  : NxD array of data samples
    #  label : Nx1 array of (continuous) targets
    # Output
    #  weight : the D-dim weight vector
    #  bias   : the scalar bias term

    # YOUR CODE HERE
    raise NotImplementedError()

    return weight, bias


def linreg(weight, bias, data, basis_fun=None):
    # Applies a linear regression model to the given data, optionally with basis function
    # Input
    #  weight : the D-dim weight vector
    #  bias   : the scalar bias term
    #  data   : NxD array of data samples
    #  basis_fun : maps an array of samples to an array of feature vectors
    # Output
    #  predicted label: Nx1 array
    #  Be aware that the output.shape should be (N, 1) instead of (N, )!

    if basis_fun is not None:
        data = basis_fun(data)
    # YOUR CODE HERE
    raise NotImplementedError()

We will need some evaluation measures to judge the performance of our models.
Implement the mean-squared error (MSE) and root mean square (RMS) functions below.

In [ ]:
def mse(pred, gt):
    # YOUR CODE HERE
    raise NotImplementedError()


def rms(pred, gt):
    # YOUR CODE HERE
    raise NotImplementedError()

Let's apply your implementation on some data!

In [ ]:
dataset_name = "sine"
train_data, train_label = load_data(f"regression_{dataset_name}_train")
test_data, test_label = load_data(f"regression_{dataset_name}_test")

In [ ]:
params = least_squares(train_data, train_label)
plot_(train_data, train_label, params)
plot_(test_data, test_label, params)

As the example shows, a purely linear model is on its own not able to model complex behavior.
However, we can again apply non-linear basis functions to the data.
Let's implement some basis function and see what effects we can observe.

Implement a polynomial basis function below.
Since we work on 1D data here, we can use a simplified formulation:

$$
\varphi_d(x) = (1, x, x^2, \ldots, x^d)^\mathsf{T}
$$

In [ ]:
def poly(x, d):
    # Input
    #  x : NxD array of data samples (In this case D=1 since input is 1-dimensional)
    #  d : degree of the polynomial
    # Output
    #  feature : Nxd array of transformed features

    # YOUR CODE HERE
    raise NotImplementedError()

We will now test how well they perform, and see which hyperparameter (i.e. the degree of the polynomial/the maximum frequency of the sinusoid) performs the best.
We also plot the norm of the parameters.

In [ ]:
def evaluate_basis_function(solver, fun, max_degree):
    error_train, error_test, weight_norm = [], [], []
    for d in range(1, max_degree):
        phi = partial(fun, d=d)
        params = solver(phi(train_data), train_label)

        # uncomment to visualize the regression results for all degrees
        plot_(train_data, train_label, params, phi)
        plot_(test_data, test_label, params, phi)

        train_pred = linreg(*params, train_data, phi)
        test_pred = linreg(*params, test_data, phi)

        error_train.append(rms(train_pred, train_label))
        error_test.append(rms(test_pred, test_label))
        weight_norm.append((params[0] ** 2).sum() + params[1] ** 2)
    plt.title(fun.__name__)
    ln1 = plt.plot(error_train, label="Train error")
    ln2 = plt.plot(error_test, label="Test error")
    plt.xlabel("Degree"), plt.ylabel("RMS")
    ax2 = plt.twinx()
    plt.ylabel("Norm of w")
    ln3 = plt.plot(weight_norm, label="weight norm", c="green")
    plt.legend(ln1 + ln2 + ln3, ["Train error", "Test error", "Weight norm"])
    plt.show()

    print(f"Best test error: {min(error_test)}")


evaluate_basis_function(least_squares, poly, 15)

In addition to the polynomial basis function, we can utilize features based on the Fourier series.
In general, we can represent a periodic signal, denoted as $s(x)$, using a series of sines and cosines as follows:

$$
s(x) = A_0 + \sum_{n=1}^\infty(A_n cos(\frac{2\pi nx}{P}) + B_n sin(\frac{2\pi nx}{P}))
$$

By utilizing the basis function defined below, we can transform our original time-series data into a combination of sine and cosine waves with different frequencies.
Subsequently, the linear regression technique can be applied to estimate the coefficients $A_0$, $A_n$ and $B_n$.

In [ ]:
def fourier(x, d):
    P = 1

    return np.concatenate(
        [
            np.sin(2 * np.pi * (i // 2) / P * x + (0.5 * np.pi * (i % 2)))
            for i in range(2 * d)
        ],
        axis=-1,
    )


evaluate_basis_function(least_squares, fourier, 15)

While the training loss decreases, the obtained result appears peculiar; in particular, we observe highly oscillating spikes, which do not match our data.
Try to find a better fit to the sampled data by adjusting the `P` value in the `fourier` function to produce a more reasonable result.

Can you explain why we observe this phenomenon, and why adjusting `P` helps to alleviate it?

# Q2: Implement regularized least-squares
While the results from fitting a regression function with higher-order polynomials look promising, we observed severe overfitting due to the small size of the training set and the large capacity of the model.
We will now implement regularization in order to avoid this behavior.

Complete the function below to compute the ridge regression solution (i.e. least-squares solution with sqaured penalty on the norm of the weight vector).
Keep in mind that we do not want to penalize the bias term.

In [ ]:
def regularized_least_squares(data, label, lam):
    # Input
    #  data  : NxD array of data samples
    #  label : Nx1 array of (continuous) targets
    #  lam   : lambda, the ridge parameter/regularization coefficient
    # Output
    #  weight : the D-dim weight vector
    #  bias   : the scalar bias term

    # YOUR CODE HERE
    raise NotImplementedError()

    return weight, bias

Let's repeat the experiment from above with the regularized least-squares solver.
What do you observe now?

In [ ]:
for lam in (0.001, 0.01, 0.1):
    print(f"##### lambda = {lam} #####")
    solver = partial(regularized_least_squares, lam=lam)
    evaluate_basis_function(solver, poly, 20)
    # evaluate_basis_function(solver, fourier, 15)

# Q3: Extending 1D Regression to 2D
In this section, we will apply the algorithm developed in the previous sections to handle 2D data. This means that our `data` will now have a shape of `(N, 2)` (`label` will still have a shape of `(N, 1)`).

To demonstrate the concept, we will utilize a 2D Gaussian probability density function (PDF) as our target function. From this target function, we will sample data points to create our input data (x, y) and corresponding labels (z) pairs. Let's visualize the target function to gain a better understanding.

In [ ]:
from matplotlib import pyplot as plt, cm
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import multivariate_normal

mu, sigma = [0, 0], [[0.1, 0], [0, 0.1]]


def plot_gt(ax=None, xlim=[-1, 1], ylim=[-1, 1]):
    x, y = np.linspace(xlim[0], xlim[1], 100), np.linspace(ylim[0], ylim[1], 100)
    X, Y = np.meshgrid(x, y)
    pos = np.stack([X, Y], axis=-1)
    rv1 = multivariate_normal(mu, sigma)
    pdf = rv1.pdf(pos)

    if ax is None:
        ax = plt.axes(projection="3d")
    surf = ax.plot_surface(X, Y, pdf, facecolors=cm.viridis(pdf / pdf.max()), alpha=0.2)
    surf.set_facecolor((0, 0, 0, 0))


plot_gt()
plt.show()

In this step, we will take some samples from this function and fit a plane to these data points.

In [ ]:
# sample n_sample points
def sample(n_sample, std):

    pos = np.random.randn(n_sample, 2) * std

    rv1 = multivariate_normal(mu, sigma)
    pdf = rv1.pdf(pos) + np.random.randn(n_sample) * 0.1
    data, label = pos.reshape(-1, 2), pdf.reshape(-1, 1)
    return data, label


def plot_result(data, label, weight, bias, fun=None):
    n_sample = int(np.sqrt(data.shape[0]))
    plt.figure()
    ax = plt.axes(projection="3d")

    n_pts = 100
    xx = np.linspace(data[:, 0].min(), data[:, 0].max(), n_pts)
    yy = np.linspace(data[:, 1].min(), data[:, 1].max(), n_pts)
    # plot hypothesis
    X, Y = np.meshgrid(xx, yy)
    pos = np.c_[X.flatten(), Y.flatten()]
    if fun is not None:
        pos = fun(pos)
    Z = (pos @ weight + bias).reshape(n_pts, n_pts)
    ax.plot_surface(X, Y, Z, color="yellow", alpha=0.5)

    # plot training data
    ax.scatter(data[:, 0], data[:, 1], label, c="b")

    # plot ground truth
    plot_gt(
        ax,
        xlim=[data[:, 0].min(), data[:, 0].max()],
        ylim=[data[:, 1].min(), data[:, 1].max()],
    )

    ax.axes.set_xlim3d(left=data[:, 0].min(), right=data[:, 0].max())
    ax.axes.set_ylim3d(bottom=data[:, 1].min(), top=data[:, 1].max())
    ax.axes.set_zlim3d(bottom=label.min(), top=label.max())
    plt.show()


np.random.seed(42)
n_sample = 100
train_data, train_label = sample(n_sample, std=0.4)
test_data, test_label = sample(n_sample * 2, std=0.3)

weight, bias = least_squares(train_data, train_label)
label_pred = linreg(weight, bias, train_data)

print("Train mean squared error :", mse(label_pred, train_label))
print("Test mean squared error :", mse(linreg(weight, bias, test_data), test_label))
plot_result(train_data, train_label, weight, bias)

As expected, a plane is unable to properly fit the given data points. Therefore, it becomes necessary to employ a non-linear basis function to transform the data.

Implement a 2D polynomial basis function `poly2d` in the code cell below.
Use the formula

$$ 
\varphi _d(x, y)_{i,j} = ( x^i y^j ) \text{ for } i = 0,...,d, j = 0, ..., d-i \\
$$

Examples:
$
\varphi _1(\mathbf{x}): ( x, y ) \mapsto (1, x, y) \\
\varphi _2(\mathbf{x}): ( x, y ) \mapsto (1, x, x^2, y, xy, ) \\
$

In [ ]:
def poly2d(x, d):
    # Input
    #  x : NxD array of data samples (In this case D=2 since input is 2-dimensional)
    #  d : degree of the polynomial

    # Output
    #  feature : N x Dout array of transformed features,
    #            where Dout = (d+1)(d+2)/2

    # YOUR CODE HERE
    feature = np.stack(
        [x[:, 0] ** i * x[:, 1] ** j for i in range(d + 1) for j in range(d + 1 - i)],
        axis=-1,
    )
    return feature


for d in [1, 2, 3, 4]:
    weight, bias = least_squares(poly2d(train_data, d), train_label)
    label_pred = linreg(weight, bias, poly2d(train_data, d))

    print("Degree :", d)
    print("Train mean squared error :", mse(label_pred, train_label))
    print(
        "Test mean squared error :",
        mse(linreg(weight, bias, poly2d(test_data, d)), test_label),
    )
    plot_result(train_data, train_label, weight, bias, partial(poly2d, d=d))
    print("-" * 50)

# Q4: Error Decompositions in the Bias-Variance Tradeoff
In this exercise, you will derive the error decomposition that we used to understand the bias-variance tradeoff.

Derive the error decomposition of the squared error into structural error and noise:

$$
\mathbb{E}[L]=\int\{y(\mathbf{x})-\mathbb{E}[t \mid \mathbf{x}]\}^2 p(\mathbf{x}) \mathrm{d} \mathbf{x}+\int\{\mathbb{E}[t \mid \mathbf{x}]-t\}^2 p(\mathbf{x}) \mathrm{d} \mathbf{x}
$$